## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuatapere,NZ,-46.13,167.68,48.99,87,16,8.99,few clouds
1,1,Hobart,AU,-42.88,147.33,53.60,87,75,4.70,broken clouds
2,2,Miri,MY,4.41,114.01,75.20,94,40,5.82,moderate rain
3,3,Agirish,RU,61.92,63.02,26.19,97,0,8.25,clear sky
4,4,Busselton,AU,-33.65,115.33,51.01,95,81,5.79,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Miri,MY,4.41,114.01,75.20,94,40,5.82,moderate rain
10,10,Padang,ID,-0.95,100.35,78.24,72,100,2.71,overcast clouds
13,13,Birao,CF,10.28,22.79,79.45,36,0,2.33,clear sky
20,20,Alvarado,MX,18.77,-95.77,82.99,46,93,23.00,light rain
31,31,Buchanan,LR,5.88,-10.05,78.42,88,78,8.43,broken clouds
33,33,Kapaa,US,22.08,-159.32,80.01,88,1,0.74,clear sky
35,35,Kahului,US,20.89,-156.47,84.20,62,20,5.82,few clouds
40,40,Bajil,YE,15.06,43.29,82.35,57,0,6.67,clear sky
45,45,Vaini,TO,-21.20,-175.20,75.20,88,75,8.05,broken clouds
46,46,Atuona,PF,-9.80,-139.03,77.99,78,10,15.39,light rain


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                209
City                   209
Country                208
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                208
City                   208
Country                208
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Miri,MY,75.20,moderate rain,4.41,114.01,
10,Padang,ID,78.24,overcast clouds,-0.95,100.35,
13,Birao,CF,79.45,clear sky,10.28,22.79,
20,Alvarado,MX,82.99,light rain,18.77,-95.77,
31,Buchanan,LR,78.42,broken clouds,5.88,-10.05,
33,Kapaa,US,80.01,clear sky,22.08,-159.32,
35,Kahului,US,84.20,few clouds,20.89,-156.47,
40,Bajil,YE,82.35,clear sky,15.06,43.29,
45,Vaini,TO,75.20,broken clouds,-21.20,-175.20,
46,Atuona,PF,77.99,light rain,-9.80,-139.03,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"].astype(bool)]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Miri,MY,75.20,moderate rain,4.41,114.01,"Mega Hotel, Miri"
10,Padang,ID,78.24,overcast clouds,-0.95,100.35,Mercure Padang
20,Alvarado,MX,82.99,light rain,18.77,-95.77,Del Pastor
31,Buchanan,LR,78.42,broken clouds,5.88,-10.05,Kamaneahn Hotel Inc
33,Kapaa,US,80.01,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
677,Puebloviejo,CO,84.99,moderate rain,10.99,-74.28,The Journey Hostel
679,Hobyo,SO,80.76,light rain,5.35,48.53,Osmani Hotel
697,Abu Dhabi,AE,82.40,clear sky,24.47,54.37,Beach Rotana
698,Santa Isabel Do Rio Negro,BR,81.37,scattered clouds,-0.41,-65.02,Hotel GSA


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))